In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = pd.read_csv("facial-emotion-recognition/facial-emotion-recognition-dataset.csv", sep=";")
image_dir = 'facial-emotion-recognition/images'



def normalize_image(image_path):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image_array = np.array(image, dtype=np.float32)
    normalized_image = image_array / 255.0  # Normalize pixel values
    return normalized_image

# Convert images into a 2D matrix where each row represents a flattened image
def images_to_matrix(image_list):
    flattened_images = [img.flatten() for img in image_list]
    return np.array(flattened_images)

# PCA using Singular Value Decomposition (SVD)
def pca_svd(images, k):
    # Step 1: Compute the mean image
    mean_image = np.mean(images, axis=0)
    
    # Step 2: Subtract the mean image from all images (mean normalization)
    centered_images = images - mean_image
    
    # Step 3: Perform SVD
    U, S, Vt = np.linalg.svd(centered_images, full_matrices=False)
    
    # Step 4: Select the top k components (eigenvectors)
    principal_components = Vt[:k]
    
    # Step 5: Project the original images onto the principal components
    projected_images = np.dot(centered_images, principal_components.T)
    
    return projected_images, principal_components

# Set up the data
sample_set_id = 0
emotion_images = ['Anger.jpg', 'Contempt.jpg', 'Disgust.jpg', 'Fear.jpg', 'Happy.jpg', 'Neutral.jpg', 'Sad.jpg', 'Surprised.jpg']
image_dir = 'facial-emotion-recognition/images'

# Load and normalize the images
normalized_images = []
for emotion in emotion_images:
    image_path = os.path.join(image_dir, f"{sample_set_id}", emotion)
    normalized_img = normalize_image(image_path)
    normalized_images.append(normalized_img)

# Convert the list of images into a 2D matrix (where each row is a flattened image)
image_matrix = images_to_matrix(normalized_images)

# Perform PCA using SVD, select top k principal components (e.g., k=2)
k = 2
projected_images, principal_components = pca_svd(image_matrix, k)

# Plot the original normalized images
fig, axes = plt.subplots(1, len(normalized_images), figsize=(12, 4))
for i, img in enumerate(normalized_images):
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(emotion_images[i])

plt.show()

# Visualize the projected images onto the first two principal components
plt.scatter(projected_images[:, 0], projected_images[:, 1], c='b', marker='o')
plt.title("Images Projected onto First Two Principal Components")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

/tmp/ipykernel_10278/2614862819.py:17: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  image_array = np.array(image, dtype=np.float32)


MemoryError: Unable to allocate 2.87 PiB for an array with shape (20113920, 20113920) and data type float64